# Zonal Averaging

In [ ]:
import uxarray as ux

uxds = ux.open_dataset(
    "../../test/meshfiles/ugrid/outCSne30/outCSne30.ug",
    "../../test/meshfiles/ugrid/outCSne30/outCSne30_vortex.nc",
)

## What is a Zonal Average/Mean?


```{see-also}
NCL Zonal Average

https://www.ncl.ucar.edu/Applications/zonal.shtml 
```

In [ ]:
zonal_mean_psi = uxds["psi"].zonal_mean()
zonal_mean_psi

In [ ]:
zonal_mean_psi.plot.line()

In [ ]:
zonal_mean_psi.latitudes

In [ ]:
uxds["psi"].zonal_mean(lat=(-20, 20, 1))

## Spherical Bounding Box

The zonal average can be computed using UXarray's robust spherical bounding box for computing intersections, which takes into account extremes along edges when they are represented as great circle arcs on the sphere. 

In [ ]:
zonal_mean_psi = uxds["psi"].zonal_mean(
    lat=(-90, 90, 5), use_spherical_bounding_box=True
)
zonal_mean_psi